## Generate function to standardize data based on different criteria

In [1]:
import sys
sys.path.append('/'.join(sys.path[0].split('/')[:-1]))

import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import time
import os
import healpy as hp

In [2]:
datadir = "../data/healpix/"
input_dir = datadir + "5.625deg_nearest/"
pred_save_path = datadir + "predictions/"

train_years = ('1979', '2012')
val_years = ('2013', '2016')
test_years = ('2017', '2018')

nodes = 12*16*16
max_lead_time = 5*24
lead_time = 6
out_features = 2
nb_timesteps = 2

obs = xr.open_mfdataset(pred_save_path + 'observations_nearest.nc', combine='by_coords')

In [3]:
chunk_size = 521

In [4]:
z500 = xr.open_mfdataset(f'{input_dir}geopotential_500/*.nc', combine='by_coords', chunks={'time':chunk_size}).rename({'z':'z500'})
t850 = xr.open_mfdataset(f'{input_dir}temperature_850/*.nc', combine='by_coords', chunks={'time':chunk_size}).rename({'t':'t850'})
rad = xr.open_mfdataset(f'{input_dir}toa_incident_solar_radiation/*.nc', combine='by_coords', chunks={'time':chunk_size})

z500 = z500.isel(time=slice(7, None))
t850 = t850.isel(time=slice(7, None))

constants = xr.open_dataset(f'{input_dir}constants/constants_5.625deg.nc').rename({'orography' :'orog'})
constants = constants.assign(cos_lon=lambda x: np.cos(np.deg2rad(x.lon)))
constants = constants.assign(sin_lon=lambda x: np.sin(np.deg2rad(x.lon)))

temp = xr.DataArray(np.zeros(z500.dims['time']), coords=[('time', z500.time.values)])
constants, _ = xr.broadcast(constants, temp)

In [5]:
ds = xr.merge([z500, t850, rad], compat='override')
#ds = xr.merge([z500, t850, orog, lats, lsm, slt, rad], compat='override')

ds_train = ds.sel(time=slice(*train_years))
ds_valid = ds.sel(time=slice(*val_years))
ds_test = ds.sel(time=slice(*test_years))

Standardize based on monthly statistics

In [6]:
aux = ds_train.groupby('time.month').mean().compute()

In [7]:
aux.to_netcdf(f'{input_dir}' + 'mean_train_features_dynamic_month.nc')

In [8]:
aux = ds_train.groupby('time.month').std().compute()
aux.to_netcdf(f'{input_dir}' + 'std_train_features_dynamic_month.nc')

In [ ]:
ds_train.groupby('time.month').std().to_netcdf(f'{input_dir}' + 'std_train_features_dynamic_month.nc')

In [6]:
z500_train = z500.sel(time=slice(*train_years))
aux_z500 = z500.groupby('time.month') - z500_train.groupby('time.month').mean()
aux_z500 = aux_z500.groupby('time.month') / z500_train.groupby('time.month').std()

In [7]:
aux_z500.compute()

<xarray.Dataset>
Dimensions:  (node: 3072, time: 350633)
Coordinates:
    level    int32 500
    lon      (node) float64 45.0 47.81 42.19 45.0 ... 315.0 317.8 312.2 315.0
    lat      (node) float64 2.388 4.78 4.78 7.181 ... -7.181 -4.78 -4.78 -2.388
  * node     (node) int64 0 1 2 3 4 5 6 7 ... 3065 3066 3067 3068 3069 3070 3071
  * time     (time) datetime64[ns] 1979-01-01T07:00:00 ... 2018-12-31T23:00:00
    month    (time) int64 1 1 1 1 1 1 1 1 1 1 ... 12 12 12 12 12 12 12 12 12 12
Data variables:
    z500     (time, node) float64 0.008527 -0.0815 -0.1541 ... 1.442 1.055 1.194

In [8]:
aux_z500.to_netcdf(input_dir +'geopotential_500/month/'+ 'monthly_standardized_data.nc')

In [14]:
t850_train = t850.sel(time=slice(*train_years))
aux_t850 = t850.groupby('time.month') - t850_train.groupby('time.month').mean()
aux_t850 = aux_t850.groupby('time.month') / t850_train.groupby('time.month').std()

In [ ]:
aux_t850.compute()

In [16]:

aux_t850.to_netcdf(input_dir +'temperature_850/month/'+ 'monthly_standardized_data.nc')

In [23]:
rad_train = rad.sel(time=slice(*train_years))
aux_rad = rad.groupby('time.month') - rad_train.groupby('time.month').mean()
aux_rad = aux_rad.groupby('time.month') / rad_train.groupby('time.month').std()


In [24]:
aux_rad.compute()
aux_rad.to_netcdf(input_dir +'toa_incident_solar_radiation/month/'+ 'monthly_standardized_data.nc')

/home/bolon/miniconda3/envs/weather/lib/python3.7/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))
/home/bolon/miniconda3/envs/weather/lib/python3.7/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))


In [ ]:
z500_train = z500.sel(time=slice(*train_years))
aux_z500 = z500.groupby('time.month') - z500_train.groupby('time.month').mean()
aux_z500 = aux_z500.groupby('time.month') / z500_train.groupby('time.month').std()

In [7]:
def get_standarization_params(ds, mean='mean', std='std', time='point'):
    """
    Compute parameters for standardizing data.
    
    Parameters
    ----------
    ds : xarray Dataset
        Dataset containing the input data
    mean: str
    std:  str
    time: str
    """
    if mean.lower() not in ['mean', 'median']:
        raise ValueError("mean parameter not valid. \
        It must be: \
        \n-'mean' or \
        \n-'median'")
    if std.lower() not in ['std','iqr']:
        raise ValueError("std parameter not valid. \
        It must be: \
        \n-'std' for Standard Deviation or \
        \n-'iqr' for Interquartile Range ")
    if time.lower() not in ['point','week', 'month']:
        raise ValueError("time parameter not valid. \
        It must be: \
        \n-'point' for statistic using complete dataset \
        \n-'week' for statistic using weekly moving window values\
        \n-'month' for statistic using monthly (30d) moving window values")
    
    if time == 'point':
        if mean == 'mean':
            ds_mean = ds.mean(('time','node')).compute()
        else:
            ds_mean = ds.median('time').mean('node').compute()
    
        if std == 'std':
            ds_std = ds.std(('time','node')).compute()
        else:
            q1, q3 = ds.quantile([0.25, 0.75], dim='time').compute().mean('node').to_array().values[0]
            ds_std = q3 - q1
    else:
        if time == 'week': t = 7  
        else: t=30
            
        if mean == 'mean':
            ds_mean = ds.rolling(time=24*t, center=True).mean().mean('node')
        else:
            ds_mean = ds.rolling(time=24*t, center=True).median().mean('node')
        if std == 'std':
            ds_std = ds.rolling(time=24*t, center=True).std().mean('node')
        else:
            quantiles = ds.rolling(time=24*t, center=True)\
            .construct('tmp').quantile([0.25, 0.75], dim='tmp').mean('node').to_array()
            ds_std = quantiles[:,1] - quantiles[:,0]
    
    return ds_mean, ds_std

In [ ]:
z500_mean, z500_std = get_standarization_params(z500, 'mean', 'std', 'point')

In [24]:
z500_mean

<xarray.Dataset>
Dimensions:  ()
Coordinates:
    level    int32 500
Data variables:
    z500     float64 5.551e+04

In [20]:
t850_mean, t850_std = get_standarization_params(t850, 'mean', 'std', 'point')

In [25]:
t850_mean

<xarray.Dataset>
Dimensions:  ()
Coordinates:
    level    int32 850
Data variables:
    t850     float64 281.1

In [22]:
toa_mean, toa_std = get_standarization_params(rad, 'mean', 'std', 'point')

In [10]:
z500_standardized = (z500 - z500_mean) / z500_std

In [9]:
z500_mean_week, z500_std_week = get_standarization_params(z500, 'mean', 'std', 'week')

In [10]:
z500_mean_month, z500_std_month = get_standarization_params(z500, 'mean', 'std', 'month')

In [11]:
z500_median, z500_iqr = get_standarization_params(z500, 'median', 'iqr', 'point')

In [12]:
z500_median_week, z500_iqr_week = get_standarization_params(z500, 'median', 'iqr', 'week')

In [13]:
z500_mean_std = (z500 - z500_mean)/z500_std

In [14]:
z500_mean_std_week = (z500 - z500_mean_week)/z500_std_week

In [15]:
z500_mean_std_week

<xarray.Dataset>
Dimensions:  (node: 3072, time: 350633)
Coordinates:
    level    int32 500
  * time     (time) datetime64[ns] 1979-01-01T07:00:00 ... 2018-12-31T23:00:00
    lat      (node) float64 dask.array<chunksize=(3072,), meta=np.ndarray>
    lon      (node) float64 dask.array<chunksize=(3072,), meta=np.ndarray>
  * node     (node) int64 0 1 2 3 4 5 6 7 ... 3065 3066 3067 3068 3069 3070 3071
Data variables:
    z500     (time, node) float64 dask.array<chunksize=(1, 3072), meta=np.ndarray>

In [16]:
z500_median_std_week = (z500 - z500_median_week)/z500_iqr_week